In [1]:
import os
import openai

In [2]:
# Read API Key
with open('API_KEY.txt') as f:
    api_key = f.readlines()[0]

# Set API Key
openai.api_key = api_key

## Steps

```
1. Ingest tabular data and convert to SQL database.
2. Ingest natural language request.
3. Convert NLP to SQL using OpenAI API.
4. Send SQL request to database
5. Return result.
```

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv('data/sales_data_sample.csv')

In [5]:
df.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


In [6]:
# What was the total sum of sales for each quarter ?
df.groupby('QTR_ID').sum()['SALES']

C:\Users\adutt\AppData\Local\Temp\ipykernel_28724\4104545415.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('QTR_ID').sum()['SALES']


QTR_ID
1    2350817.73
2    2048120.30
3    1758910.81
4    3874780.01
Name: SALES, dtype: float64

## Create SQL Database in Memory

In [7]:
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text

In [8]:
# Setup TEMP DB in RAM
temp_db = create_engine('sqlite:///:memory:', echo=True)

In [9]:
# Push Pandas DF to TEMP DB
data = df.to_sql(name='Sales', con=temp_db)

2023-03-17 11:36:45,727 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 11:36:45,728 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Sales")
2023-03-17 11:36:45,729 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 11:36:45,730 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Sales")
2023-03-17 11:36:45,731 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 11:36:45,732 INFO sqlalchemy.engine.Engine ROLLBACK
2023-03-17 11:36:45,735 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 11:36:45,736 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Sales" (
	"index" BIGINT, 
	"ORDERNUMBER" BIGINT, 
	"QUANTITYORDERED" BIGINT, 
	"PRICEEACH" FLOAT, 
	"ORDERLINENUMBER" BIGINT, 
	"SALES" FLOAT, 
	"ORDERDATE" TEXT, 
	"STATUS" TEXT, 
	"QTR_ID" BIGINT, 
	"MONTH_ID" BIGINT, 
	"YEAR_ID" BIGINT, 
	"PRODUCTLINE" TEXT, 
	"MSRP" BIGINT, 
	"PRODUCTCODE" TEXT, 
	"CUSTOMERNAME" TEXT, 
	"PHONE" TEXT, 
	"ADDRESSLINE1" TEXT, 
	"ADDRESSLINE2" TEXT, 
	"CITY" TEXT, 
	"STATE" TEXT, 
	

In [10]:
# Run SQL Queries on TEMP DB
# Make the connection to TEMP DB and run SQL Query
with temp_db.connect() as conn:
    result = conn.execute(text("SELECT SUM(SALES) FROM Sales"))

2023-03-17 11:36:45,851 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 11:36:45,852 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) FROM Sales
2023-03-17 11:36:45,853 INFO sqlalchemy.engine.Engine [generated in 0.00218s] ()
2023-03-17 11:36:45,855 INFO sqlalchemy.engine.Engine ROLLBACK


In [11]:
result.all()

[(10032628.85000001,)]

## Ingest Natural Language Request (Prompt Engineering)

Steps:

```
1. Create a table structure definition from Pandas dataframe above to send to OpenAI API.
2. Grab NLP input from user for querying SQL Database.
3. Combine prompts for OpenAI API call
```

In [12]:
def create_table_def(df):
    prompt = f"""### sqlite SQL table, with its properties:
    #
    # Sales({",".join(str(col) for col in df.columns)})
    # 
    """
    
    return prompt

In [13]:
print(create_table_def(df))

### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    # 
    


In [14]:
def prompt_input():
    nlp_txt = input("Enter your query: ")
    return nlp_txt

In [15]:
prompt_input()

Enter your query:  return the sum of SALES per POSTALCODE


'return the sum of SALES per POSTALCODE'

In [16]:
def combine_prompts(df, query_prompt):
    definition = create_table_def(df)
    query_init_string = f'### A query to answer: {query_prompt}\nSELECT'
    return definition + query_init_string

In [17]:
# TEST
nlp_text = prompt_input()
print(combine_prompts(df, nlp_text))

Enter your query:  return the sum of SALES per POSTALCODE


### sqlite SQL table, with its properties:
    #
    # Sales(ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,PRODUCTLINE,MSRP,PRODUCTCODE,CUSTOMERNAME,PHONE,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE)
    # 
    ### A query to answer: return the sum of SALES per POSTALCODE
SELECT


## Convert NLP to SQL Query using OpenAI API

Completion API Types:
```
1. Text Completion (GPT-3.5)
2. Code Completion (Codex)
```

In [19]:
response = openai.Completion.create(
    model='code-davinci-002',
    prompt=combine_prompts(df, nlp_text),
    temperature=0,
    max_tokens=150,
    top_p=1.0,
    frequency_penalty=0,
    presence_penalty=0,
    stop=['#', ';']
)

In [20]:
response

<OpenAIObject text_completion id=cmpl-6v9Fi0PCX5bqjgS1JaSOJXe1l26rK at 0x274ba9e0cc0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": " POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE"
    }
  ],
  "created": 1679078282,
  "id": "cmpl-6v9Fi0PCX5bqjgS1JaSOJXe1l26rK",
  "model": "code-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 19,
    "prompt_tokens": 163,
    "total_tokens": 182
  }
}

In [21]:
# Cleaned up Response
response['choices'][0]['text']

' POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

## Return Result

In [22]:
def handle_response(response):
    query = response['choices'][0]['text']
    if query.startswith(' '):
        query = "SELECT" + query
    return query

In [23]:
# Final Response
handle_response(response)

'SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE'

In [24]:
with temp_db.connect() as conn:
    result = conn.execute(text(handle_response(response)))

2023-03-17 11:40:26,921 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 11:40:26,923 INFO sqlalchemy.engine.Engine SELECT POSTALCODE, SUM(SALES) FROM Sales GROUP BY POSTALCODE
2023-03-17 11:40:26,924 INFO sqlalchemy.engine.Engine [generated in 0.00259s] ()
2023-03-17 11:40:26,926 INFO sqlalchemy.engine.Engine ROLLBACK


In [25]:
result.all()

[(None, 272407.14),
 ('10022', 560787.7699999998),
 ('10100', 94117.26000000002),
 ('106-0032', 120562.73999999996),
 ('1203', 117713.55999999998),
 ('1227 MM', 94015.73),
 ('13008', 74936.14),
 ('1734', 145041.6),
 ('2', 57756.43),
 ('2060', 153996.13000000003),
 ('2067', 151570.98000000004),
 ('21240', 111250.37999999996),
 ('24067', 85555.98999999998),
 ('24100', 137955.72000000003),
 ('28023', 170257.33000000005),
 ('28034', 912294.1100000002),
 ('3004', 200995.40999999997),
 ('31000', 70488.44),
 ('3150', 64591.46000000001),
 ('4101', 59469.11999999999),
 ('4110', 116599.19),
 ('41101', 54723.62),
 ('42100', 142601.33000000002),
 ('44000', 204304.86),
 ('5020', 149798.63),
 ('50553', 207874.86),
 ('50739', 100306.58),
 ('51003', 154069.65999999997),
 ('51100', 135042.94),
 ('51247', 139243.99999999994),
 ('530-0003', 67605.07),
 ('58339', 165255.20000000004),
 ('59000', 69052.41),
 ('60528', 85171.58999999998),
 ('62005', 131685.30000000002),
 ('67000', 80438.48),
 ('69004', 14287